### Note: The dataset used in this project is confidential and proprietary to the company. As such, it is not included in this repository.

In [ ]:
# Cell 1: Mount Google Drive and Install Dependencies
from google.colab import drive
drive.mount('/content/drive')

%pip install ultralytics
%pip install tqdm
%pip install PyYAML
%nvidia-smi  # Verify GPU availability

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Thu Jan 16 19:38:11 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                            

In [ ]:
# Cell 2: Import Required Libraries
import os
import yaml
import cv2
import torch
from ultralytics import YOLO
from tqdm.notebook import tqdm
import logging
import shutil
from pathlib import Path
from datetime import datetime

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


In [ ]:
# Cell 3: Configuration
class Config:
    # Update paths for Google Drive
    DRIVE_ROOT = "/content/drive/MyDrive"
    DATA_DIR = os.path.join(DRIVE_ROOT, "VTM_Object_Detection_Data_Set")
    OUTPUT_DIR = os.path.join(DRIVE_ROOT, "VTM_output")
    WEIGHTS_DIR = os.path.join(DRIVE_ROOT, "VTM_weights")
    MODEL_NAME = "yolov8n.pt"
    EPOCHS = 100
    IMG_SIZE = 640
    BATCH_SIZE = 16  # Adjust based on GPU memory

    CLASSES = [
        'AWB-No', 'FOB-Value-in-INR', 'Goods-Description',
        'Non-GST-Invoice-No', 'Qty', 'Total-Value', 'Unit-Value'
    ]

    @classmethod
    def create_dirs(cls):
        os.makedirs(cls.OUTPUT_DIR, exist_ok=True)
        os.makedirs(cls.WEIGHTS_DIR, exist_ok=True)
        logger.info(f"Created output directories: {cls.OUTPUT_DIR}, {cls.WEIGHTS_DIR}")


In [ ]:
# Cell 4: Verify Dataset Structure
def verify_dataset():
    """Verify the dataset structure in Google Drive"""
    required_dirs = [
        os.path.join(Config.DATA_DIR, 'train', 'images'),
        os.path.join(Config.DATA_DIR, 'train', 'labels'),
        os.path.join(Config.DATA_DIR, 'valid', 'images'),
        os.path.join(Config.DATA_DIR, 'valid', 'labels'),
        os.path.join(Config.DATA_DIR, 'test', 'images'),
        os.path.join(Config.DATA_DIR, 'test', 'labels')
    ]

    for directory in required_dirs:
        if not os.path.exists(directory):
            raise FileNotFoundError(f"Required directory not found: {directory}")

    logger.info("Dataset structure verified successfully")

In [ ]:
# Cell 5: Create Dataset Configuration
def create_dataset_yaml():
    data_yaml = {
        'path': Config.DATA_DIR,
        'train': os.path.join('train', 'images'),
        'val': os.path.join('valid', 'images'),
        'test': os.path.join('test', 'images'),
        'nc': len(Config.CLASSES),
        'names': Config.CLASSES
    }

    yaml_path = os.path.join(Config.DATA_DIR, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)

    logger.info(f"Created dataset configuration at {yaml_path}")
    return yaml_path

In [ ]:
# Cell 6: Model Training with GPU
def train_model(yaml_path):
    logger.info("Starting model training with GPU...")

    # Verify GPU availability
    if not torch.cuda.is_available():
        logger.warning("GPU not available, falling back to CPU")
    else:
        logger.info(f"Using GPU: {torch.cuda.get_device_name(0)}")

    # Initialize model
    model = YOLO(Config.MODEL_NAME)

    # Train the model
    try:
        results = model.train(
            data=yaml_path,
            epochs=Config.EPOCHS,
            imgsz=Config.IMG_SIZE,
            batch=Config.BATCH_SIZE,
            project=Config.WEIGHTS_DIR,
            name=f'field_detection_{datetime.now().strftime("%Y%m%d_%H%M%S")}',
            device=0  # Use GPU
        )

        # Get path to best weights
        weights_path = os.path.join(Config.WEIGHTS_DIR,
                                  list(filter(lambda x: 'field_detection' in x,
                                            os.listdir(Config.WEIGHTS_DIR)))[-1],
                                  'weights',
                                  'best.pt')

        logger.info(f"Model training completed successfully. Weights saved at: {weights_path}")
        return weights_path
    except Exception as e:
        logger.error(f"Error during training: {str(e)}")
        raise

In [ ]:
# Cell 7: Field Detection and Cropping
class FieldDetector:
    def __init__(self, weights_path):
        self.model = YOLO(weights_path)
        self.model.to('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Loaded model from {weights_path}")

    def process_image(self, image_path):
        # Create output directory for crops
        output_dir = os.path.join(Config.OUTPUT_DIR, Path(image_path).stem)
        os.makedirs(output_dir, exist_ok=True)

        # Read image
        image = cv2.imread(image_path)
        if image is None:
            logger.error(f"Failed to read image: {image_path}")
            return

        # Perform detection
        results = self.model(image)[0]

        # Process each detection
        for i, det in enumerate(results.boxes.data):
            x1, y1, x2, y2, conf, cls = det

            if conf < 0.25:  # Confidence threshold
                continue

            # Convert coordinates to integers
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Crop field
            field_crop = image[y1:y2, x1:x2]

            # Save cropped field
            class_name = Config.CLASSES[int(cls)]
            output_path = os.path.join(output_dir, f"{class_name}.jpg")
            cv2.imwrite(output_path, field_crop)

            logger.info(f"Saved {class_name} crop to {output_path}")

        return output_dir

In [ ]:
# Cell 8: Main Execution
def main():
    # Verify dataset structure
    verify_dataset()

    # Create necessary directories
    Config.create_dirs()

    # Create dataset configuration
    yaml_path = create_dataset_yaml()

    # Train model
    weights_path = train_model(yaml_path)

    # Initialize detector
    detector = FieldDetector(weights_path)

    # Process test images
    test_dir = os.path.join(Config.DATA_DIR, 'test', 'images')
    test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir)
                  if f.endswith(('.jpg', '.png', '.jpeg'))]

    for image_path in tqdm(test_images, desc="Processing test images"):
        output_dir = detector.process_image(image_path)
        logger.info(f"Processed {image_path} -> {output_dir}")

In [ ]:
# Cell 9: Single Image Processing Function
def process_single_image(image_path, weights_path=None):
    """
    Process a single image using the trained model.

    Args:
        image_path (str): Path to the input image
        weights_path (str, optional): Path to model weights. If None, uses the latest trained weights.
    """
    if weights_path is None:
        # Find the latest weights directory
        weight_dirs = [d for d in os.listdir(Config.WEIGHTS_DIR)
                      if 'field_detection' in d]
        if not weight_dirs:
            logger.error("No trained weights found")
            return

        latest_dir = sorted(weight_dirs)[-1]
        weights_path = os.path.join(Config.WEIGHTS_DIR,
                                  latest_dir,
                                  'weights',
                                  'best.pt')

    if not os.path.exists(weights_path):
        logger.error(f"Weights file not found: {weights_path}")
        return

    detector = FieldDetector(weights_path)
    output_dir = detector.process_image(image_path)

    logger.info(f"Processing completed. Crops saved to: {output_dir}")
    return output_dir

In [ ]:
# Cell 10: Example Usage
if __name__ == "__main__":
    # For training and processing all test images
    main()

100%|██████████| 6.25M/6.25M [00:00<00:00, 268MB/s]


Ultralytics 8.3.62 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/VTM_Object_Detection_Data_Set/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/VTM_weights, name=field_detection_20250116_194004, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, s

100%|██████████| 755k/755k [00:00<00:00, 113MB/s]


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 358MB/s]


...


100 epochs completed in 0.373 hours.
Optimizer stripped from /content/drive/MyDrive/VTM_weights/field_detection_20250116_194004/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/VTM_weights/field_detection_20250116_194004/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/VTM_weights/field_detection_20250116_194004/weights/best.pt...
Ultralytics 8.3.62 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.08it/s]


                   all        113        791      0.928          1      0.995      0.896
                AWB-No        113        113      0.509          1      0.995      0.682
      FOB-Value-in-INR        113        113          1          1      0.995      0.928
     Goods-Description        113        113          1          1      0.995      0.929
    Non-GST-Invoice-No        113        113      0.996          1      0.995      0.928
                   Qty        113        113      0.996          1      0.995       0.91
           Total-Value        113        113      0.996          1      0.995      0.968
            Unit-Value        113        113      0.996          1      0.995      0.931
Speed: 0.8ms preprocess, 2.9ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to /content/drive/MyDrive/VTM_weights/field_detection_20250116_194004


Processing test images:   0%|          | 0/56 [00:00<?, ?it/s]


0: 640x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-Value, 1 Unit-Value, 10.8ms
Speed: 3.0ms preprocess, 10.8ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-Value, 1 Unit-Value, 13.5ms
Speed: 3.5ms preprocess, 13.5ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-Value, 1 Unit-Value, 14.3ms
Speed: 3.3ms preprocess, 14.3ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-Value, 1 Unit-Value, 14.7ms
Speed: 3.3ms preprocess, 14.7ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-